# t-SNE for data exploration

The algorithm (t-Distributed Stochastic Neighbor Embedding) can be used to describe high dimensional relationships in a lower dimension. This can be useful in data exploration for identifying things like clusters or other patterns in the data representation. The representation here will be vectors found by FastText for documents in a set.

The data will be decisions (uitspraken) put out by the Dutch Supreme Court (De Hoge Raad). These documents are available freely online and have already been labeled for some information on the type of case being handled and which branch of the judicial system handled the case.

### SETUP
Change directory to the `src` directory because most of the paths currently assume that location.

In [1]:
%cd ../..

/Users/dvalpey/practice/nlp_experiments


Imports for running the following code. The `ipywidgets` package is what allows us to do interactive refactoring for the t-SNE exploration.

In [2]:
from collections import Counter
from time import time

import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

from src.uitspraak_files.extract_from_xmls import parse_xmls
%matplotlib inline

ModuleNotFoundError: No module named 'gensim.summarization'

Load and parse XML files. We drop the text field here because we won't need it, and it makes the objects easier to read if we want to later.

In [ ]:
parsed_docs = list(parse_xmls())
for doc in parsed_docs:
    del doc["text"]
parsed_docs[0]

This data set has multiple categorical fields that could be used for classification. For the sake of simplicity, we'll pick on that has a smaller number of possible label (for example: subject or procedure). The legal domain in this case identifies the kind of law that applies and was taken from the more general part of the subject field. Further exploration could include coloring the data to match on of the other fields and see how the patterns change.

In [ ]:
Counter(doc.get('legal_domain') for doc in parsed_docs)

### Simple application
The t-SNE solution in Scikit Learn is easy to apply and display. It only takes about one line of code to learn and apply a lower dimenional representation of the data. In this example we will take 100-dimensional word vectors and display them in a 2-dimensional grid. The colors will represent a classification on the type of case being represented.

First, we'll plot a simple PCA and t-SNE together so we can compare the results of the different distributions.

In [ ]:
inputs = [doc['doc_vector'] for doc in parsed_docs]
labels = [doc.get('legal_domain') for doc in parsed_docs]

pca0 = time()
X_pca = TruncatedSVD(n_components=2).fit_transform(inputs)
pca_time = time() - pca0

tsne = TSNE(n_components=2, init='pca', random_state=0)
tsne0 = time()
X_tsne = tsne.fit_transform(inputs)
tsne_time = time() - tsne0

In [ ]:
pca_xy = list(zip(*X_pca))
tsne_xy = list(zip(*X_tsne))

classes = ['Strafrecht', 'Civiel recht', 'Bestuursrecht', 'Internationaal publiekrecht']
color_names = ListedColormap(['blue', 'orange', 'green', 'purple'])
color_mapping = {'Strafrecht': 0,
                 'Civiel recht': 1,
                 'Bestuursrecht': 2,
                 'Internationaal publiekrecht': 3}
colors = [color_mapping[doc.get('legal_domain')] for doc in parsed_docs]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

scatter1 = ax1.scatter(tsne_xy[0], tsne_xy[1], c=colors, cmap=color_names, marker=".")
ax1.set_title(f"t-SNE embedding of documents (time={tsne_time:.2f}s)")

scatter2 = ax2.scatter(pca_xy[0], pca_xy[1], c=colors, cmap=color_names, marker=".")
ax2.set_title(f"PCA embedding of documents (time={pca_time:.2f}s)")
fig.legend(handles=scatter2.legend_elements()[0], labels=classes, loc="center right")
plt.show()

The figure on the left (t-SNE) gives more of a spread to the data. The algorithm tends to make sparse clusters more dense and do the oposite to dense clusters, so the shapes don't have a lot of meaning in themselves - we can't really interpret the size or distance between groups directly.

We can see areas of clear separation or mixing and see that some parts of the group are more connected to groupings of another class. In this case, it does appear that there is a certain amount of mixing between the groups.

### 3d views
From the above image, it looks like there might not be a clear partition of the data. It's hard to tell if the colored sections truly merge, or if they might be superimposed over each when viewed from a higher dimensional space.
It is possible to rotate this image (disabled because it was too much for my machine at this scale) and see the data separated into colored bands in the 3rd dimension.

In [ ]:
tsne_3d = TSNE(n_components=3, init='pca', random_state=0, n_jobs=-1)
tsne0_3d = time()
X_tsne_3d = tsne_3d.fit_transform(inputs)
tsne_time_3d = time() - tsne0_3d

tsne_xyz = list(zip(*X_tsne_3d))
classes = ['Strafrecht', 'Civiel recht', 'Bestuursrecht', 'Internationaal publiekrecht']
color_names = ListedColormap(['blue', 'orange', 'green', 'purple'])
color_mapping = {'Strafrecht': 0,
                 'Civiel recht': 1,
                 'Bestuursrecht': 2,
                 'Internationaal publiekrecht': 3}
colors = [color_mapping[doc.get('legal_domain')] for doc in parsed_docs]

fig_3d = plt.figure(figsize=(15.0, 15.0))
ax_3d = fig_3d.add_subplot(111, projection='3d')

scatter_3d = ax_3d.scatter(xs=tsne_xyz[0], ys=tsne_xyz[1], zs=tsne_xyz[2], 
                        c=colors, cmap=color_names, marker=".", depthshade=True)
fig.legend(handles=scatter_3d.legend_elements()[0], labels=classes, loc="center right")
plt.show()

### t-SNE + PCA
One problem with t-SNE is that it scales quadratically with the number of features (O(n^2)). So feature reduction with this method works best with a lower number of features. It turns out that ~50 is a good number. It also happens that we can use another method like PCA to reduce our feature space before applying t-SNE, and this won't even have much effect on the final view of the data.

Below is a comparison of the results of t-SNE by itself (from above) and the combined results of t-SNE + PCA.

In [ ]:
pca0_50 = time()
X_pca_50 = PCA(n_components=50).fit_transform(inputs)
pca_time_50 = time() - pca0

tsne_50 = TSNE(n_components=2, init='pca', random_state=0, n_jobs=-1)
tsne0_50 = time()
X_tsne_50 = tsne_50.fit_transform(X_pca_50)
tsne_time_50 = time() - tsne0_50

tsne_xy_50 = list(zip(*X_tsne_50))

classes = ['Strafrecht', 'Civiel recht', 'Bestuursrecht', 'Internationaal publiekrecht']
color_names = ListedColormap(['blue', 'orange', 'green', 'purple'])
color_mapping = {'Strafrecht': 0,
                 'Civiel recht': 1,
                 'Bestuursrecht': 2,
                 'Internationaal publiekrecht': 3}
colors = [color_mapping[doc.get('legal_domain')] for doc in parsed_docs]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), sharey=True)

scatter1 = ax1.scatter(tsne_xy[0], tsne_xy[1], c=colors, cmap=color_names, marker=".")
ax1.set_title(f"t-SNE embedding of documents (time={tsne_time:.2f}s)")

scatter2 = ax2.scatter(tsne_xy_50[0], tsne_xy_50[1], c=colors, cmap=color_names, marker=".")
ax2.set_title(f"PCA reduction of documents before t-SNE (time={tsne_time_50:.2f}s)")
fig.legend(handles=scatter2.legend_elements()[0], labels=classes, loc="lower right")
plt.show()

In [ ]:
print(tsne_time, tsne_time_50)

The results look very similar, but the time taken to process the t-SNE reduction of the data is significantly (~30 seconds) faster. This will speed up our exploration of the hyperparameters for t-SNE by a lot, but give us very similar results.

### Interactive exploration
So great, in a short amount of time we can have several pretty looking plots of our data. What's wrong with that?

We've barely touched any of the pramaters for the algorithm, and these can have a huge impact on what the results look like.
* n_components: int, optional (default: 2)
* perplexity: float, optional (default: 30)
* early_exaggeration: float, optional (default: 12.0)
* learning_rate: float, optional (default: 200.0)
* n_iter: int, optional (default: 1000)

For one thing, understanding the results often requires viewing the data after applying multiple perplexity values. The best way to learn to read t-SNE plots is to try out a number of different parameters and see what effect they have. It is possible that our data would separate clearly even in a 2d plot given the right combination of parameters. 

Perplexity affects the influence that local and global distance have over the final results, and can completely change the representation. Rather than change these parameters individually, we can do a crossvalidation and plot all the results, picking one that shows the divisions we're looking for in the data. Or we can use interactive widgets to iterate through variations in our parameters.

For this, we need to define a function that exposes all the parameters we would like to explore. The PCA step will be included before this step to reduce the time needed to retrain after each change. For now, we will just add perplexity and the number of iterations (n_iter) for this example.

In [ ]:
pca0_interactive = time()
X_pca_interactive = PCA(n_components=50).fit_transform(inputs)
pca_time_interactive = time() - pca0_interactive

classes = ['Strafrecht', 'Civiel recht', 'Bestuursrecht', 'Internationaal publiekrecht']
color_names = ListedColormap(['blue', 'orange', 'green', 'purple'])
color_mapping = {'Strafrecht': 0,
                 'Civiel recht': 1,
                 'Bestuursrecht': 2,
                 'Internationaal publiekrecht': 3}
colors = [color_mapping[doc.get('legal_domain')] for doc in parsed_docs]


In [ ]:
@interact(data=widgets.fixed(X_pca_interactive),
         perplexity=widgets.IntSlider(min=5, max=100, step=5, value=30),
         n_iter=widgets.IntSlider(min=500, max=5000, step=50, value=500)
         )
def explore_tsne(data, perplexity, n_iter):
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter,
                init='pca', random_state=0, n_jobs=-1)
    tsne0 = time()
    X_tsne = tsne.fit_transform(data)
    tsne_time = time() - tsne0

    tsne_xy = list(zip(*X_tsne))

    fig = plt.figure(figsize=(15.0, 15.0))
    ax = fig.add_subplot(111)
    ax.set_title(f"t-SNE embedding of documents (time={tsne_time:.2f}s)")

    scatter = ax.scatter(tsne_xyz[0], tsne_xyz[1], 
                         c=colors, cmap=color_names, marker=".")
    fig.legend(handles=scatter.legend_elements()[0], labels=classes, loc="lower right")
    plt.show()

As a reminder, below is the default version of this plot.

In [ ]:
explore_tsne(X_pca_interactive, 30, 1000)

#### Some references
t-SNE:
* https://www.oreilly.com/content/an-illustrated-introduction-to-the-t-sne-algorithm/
* https://distill.pub/2016/misread-tsne/

ipywidgets:
* https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
* https://ipywidgets.readthedocs.io/en/stable/index.html